# **Ngram model을 이용한 문장 자동 생성기**
목표 : Ngram 모델을 이용하여 문장을 자동으로 생성하는 모델을 만들어보자.

계기 : 자연어처리 시간에 배운 Ngram 모델이 흥미로워서 실제로 사용해보고 싶었고, 인터넷에 있는 좋은 참고자료들이 많아, 이를 읽고 ChatGPT를 활용하여 한국어 문장 자동 생성기를 만들게 되었습니다.

---

※ 본 프로젝트는 아래의 코드를 참고하여 구현되었으며, 일부 구조를 유지하고 있습니다.

※ 어떠한 상업적인 목적 없이, 순수한 학습 목적의 개인 프로젝트임을 밝힙니다.

※ Smoothing 적용, Perplexity 개선 및 한국어 처리를 중심으로 코드를 구성하였습니다.

참고 자료 :
- Joshua Loehr님의 N-gram Language Model 구현 :
https://github.com/joshualoehr/ngram-language-model/blob/master/language_model.py
- 고려대학교 NLP & AI Lab : https://github.com/nlpai-lab/nlp-bible-code/blob/master/09%EC%9E%A5_%EC%96%B8%EC%96%B4%20%EB%AA%A8%EB%8D%B8/%5B9-1%5D_N_gram_%EC%96%B8%EC%96%B4_%EB%AA%A8%EB%8D%B8%EB%A1%9C_%EB%AC%B8%EC%9E%A5_%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0.ipynb




### **Ngram 모델이란?**
- N-gram 언어 모델은 앞의 N-1개의 단어가 주어졌을 때, 다음 단어가 나올 확률을 기반으로 문장을 생성하거나 평가합니다.

- n이 커질수록 문맥을 더 많이 반영할 수 있지만, 그만큼 학습 데이터에서 등장하지 않은 조합이 많아지게 됩니다.

- 이처럼 학습데이터에 없는 n-gram으로 인해 등장 확률이 0이 되는 문제를 방지하기 위해 **smoothing**을 적용합니다.
  - Smoothing이란?
    - 학습 데이터에 없는 경우에도 확률이 0이 되지 않도록 아주 작은 값을 더해주는 방법입니다.

    - Laplace smoothing, KneserNey smoothing, KatzBackoff smoothing 등이 있습니다.

  - Laplace Smoothing (Add-1)
    - 가장 기본적인 방법으로, 모든 조합 등장 횟수에 1을 더하는 방식입니다.
    - 실제 count가 0이더라도 1회 등장한 것처럼 더합니다.
    - 꼭 1이 아니라, 0.01, 0.05 와 같은 작은 값을 더할 수도 있는데, 이를 Add-λ Smoothing이라고 합니다.


- nltk 라이브러리를 활용하여 N-gram 생성을 간편하게 처리할 수 있습니다.

In [30]:
## 실습 환경 세팅
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.probability import FreqDist
import numpy as np
import codecs
from tqdm import tqdm
import random
import math

In [2]:
## 한국어 처리를 위한 사전작업
!pip3 install JPype1-py3
!pip3 install konlpy
from konlpy.tag import Okt

# nltk를 사용을 위하여 선행 패키지를 설치한다.
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for JPype1-py3: filename=JPype1_py3-0.5.5.4-cp311-cp311-linux_x86_64.whl size=3259382 sha256=e5766e1532d1f0a4d6ca384efddef1dba1747045ac5676f6495554ef2cffad10
  Stored in directory: /root/.cache/pip/wheels/c3/a6/b5/d0acc5a6e1622b48518a0ac7266a98778336a0621b532e8f06
Successfully built JPype1-py3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 42.6 MB/s eta 0:00:00


In [23]:
## 0. 데이터 다운로드
# 문장 생성을위하여 네이버 영화 리뷰 데이터셋을 다운로드한다.
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

CPU times: user 3.43 ms, sys: 18 µs, total: 3.45 ms
Wall time: 103 ms


In [24]:
## 1. 데이터로딩
# 다운로드 받은 데이터셋에서 텍스트 부분만 가져온다.
# codecs 패키지는 대용량 파일을 조금씩 읽을 수 있게 해준다.
def load_korean_data(path):
  with codecs.open(path, encoding="utf-8") as f:
    data = [line.split('\t') for line in f.read().splitlines()] # \n 제외
    data = data[1:] # header 제외
    docs = [row[1] for row in data]

    return docs # text 데이터

In [42]:
## 2. 토크나이징
tagger = Okt()
def tokenize(text):
  tokens = ['<s>'] + ['/'.join(t) for t in tagger.pos(text)] + ['</s>']
  return tokens

def preprocess(sentences, n):
    tokens = []
    for sentence in sentences:
        toks = tokenize(sentence)  # 먼저 형태소 분석
        tokens.extend(['<s>'] * (n - 1) + toks + ['</s>'])  # 문장 단위로 padding
    return tokens

In [53]:
## 3. Language Model 클래스
class LanguageModel:
  def __init__(self, train_data, n=2, laplace=1.0):
    self.n = n # 차수
    self.laplace = laplace
    self.tokens = preprocess(train_data, n)
    self.vocab = set(self.tokens)
    self.vocab_size = len(self.vocab)
    self.model = self._create_model()

  def _create_model(self):
    if self.n == 1:             # 만약 unigram이라면
      total = len(self.tokens)  # 전체 단어 개수에서
      freq = FreqDist(self.tokens) # 단어 하나하나의 등장만 계산
      return {(w, ): freq[w] / total for w in freq}

    else:
      return self._smooth()

  def _smooth(self):
    '''
    토큰 시퀀스 = ['<s>', '재미/Noun', '있다/Adjective', </s>']
    n_grams = ('재미/Noun', '있다/Adjective')
    m_grams = ('재미/Noun')

    Laplace smoothing:
      p(있다 | 재미) = (count(재미, 있다) + L) / (count(재미) + L * V(고유개수))
    '''
    n_grams = list(ngrams(self.tokens, self.n))
    m_grams = list(ngrams(self.tokens, self.n - 1)) # ngram에서 마지막 단어만 제외한 (n-1) gram을 생성합니다. (재미, 있다.) -> (재미)

    n_vocab = FreqDist(n_grams)
    m_vocab = FreqDist(m_grams)

    def smoothed_prob(n_gram):
      m_gram = n_gram[:-1] # 앞 부분
      n_count = n_vocab[n_gram]
      m_count = m_vocab[m_gram] # 앞 부분의 등장 횟수
      return (n_count + self.laplace) / (m_count + self.laplace * self.vocab_size) # (전체 등장 횟수에 라플라스 더한 값) / (앞 부분의 등장횟수 + 라플라스 * 고유개수)

    return {ng : smoothed_prob(ng) for ng in n_vocab} # 확률 분포로 만든다. ex: (있다. | 재미 )) : 0.0032

  '''
  Laplace와 다른 점이 무엇인가?
  1. Laplace를 적용하여 해결되는 경우 :
    학습에 (n-1) gram은 있었지만, 그에 이어지는 특정 단어가 없었던 경우

    (예) (재미/Noun)와 (재미, 있다)는 있으나, (재미, 귀엽다)는 없는 경우 -> Laplace가 적용되어서 아주 작은 값이 나온다.

  2. 아래의 함수를 적용하여 해결되는 경우 :
   학습 데이터에 "초콜릿" 자체가 한 번도 안 나온 경우
   그러면 n-1 자체가 없기 때문에 Laplace를 적용해도 불안정할 수 있다.

  그러므로 oov 까지 적용하여 안정적인 모델을 만들고자 한다.
  '''
  def _convert_oov(self,ngram): # 단어 미등록 문제 해결 && 학습에 존재하는 일반적인 ngram일 경우에 그대로 반환
    ngram = (ngram, ) if isinstance(ngram, str) else ngram # n =1 일 때 tuple로바꿔준다.
    masks = list(reversed(list(product((0,1), repeat=self.n)))) # 어떤 위치의 단어를 <UNK>로 바꿀지 경우의 수 계산
    for mask in masks:
      alt = tuple(token if m else "<UNK>" for token, m in zip(ngram, mask)) # <UNK>로 토큰을 치환한 후에 모델에 존재하는 n-gram을 찾는다.
      if alt in self.model:
        return alt

    return ngram

  # 문장의 혼란도를 계산한다.
  def perplexity(self, sentences):
    '''
    문제점 :
    generate_sentence_from_seed에서 return을 self.perplexity([sentence])로 받았다.
    그러나 이렇게 받으면 문제가 생길 수 있다.
    문장을 생성한 후 다시 preprocess를 적용하면, 이미 형태소 분석이 된 token을 다시 분석하게 되어 형태가 깨질 수 있다.
    예) '있다/Adjective' -> 다시 분석되며 '<s>', '있다', 'Adjective'처럼 잘못 나뉠 수 있음.

    -> 모델에 존재하지 않은 n-gram이 생기게 되어 확률이 매우 작아지게 되고, Perplexity가 비정상적(48000 등)으로 커지는 문제가 발생하였다.
    '''
    test_tokens = preprocess(sentences, self.n)
    test_ngrams = list(ngrams(test_tokens, self.n))
    N = len(test_ngrams)

    '''
    가장 잘 나타내는 ngram tuple을 찾아 반환
    -> self.model 딕셔너리에서 .get()을 통해 해당 n-gram의 확률을 가져온다.
    -> perplexity를 로그 확률로 평균을 내어 계산한다.
    -> 값이 낮을수록 더 자연스러운 문장이다.
    '''

    probs = [self.model.get(self._convert_oov(ng), 1e-8) for ng in test_ngrams] # self.model의 딕셔너리에서 .get(key, default)를 통해 확률을 불러온다.
    return math.exp(-sum(map(math.log, probs)) /N)

  # 개선 함수 : 이미 토큰화된 결과를 그대로 받아 Perplexity 계산
  def perplexity_from_tokens(self, tokens):
    # preprocess된 token 리스트를 그대로 입력받아 평가
    test_ngrams = list(ngrams(tokens, self.n))
    N = len(test_ngrams)
    probs = [self.model.get(self._convert_oov(ng), 1e-8) for ng in test_ngrams]
    return math.exp(-sum(map(math.log, probs)) / N)


  def generate_sentence_from_seed(self, seed=None, min_len = 10, max_len = 30, debug = False):
    if seed is not None:
      random.seed(seed)

    # 언어 모델에서 가장 확률이 높은 n-gram 조합을 이용하여 한 단어씩 생성한다.
    sent = ["<s>"] * max(1, self.n - 1)# 문장 시작 시에 앞의 단어가 없으므로 <s> 토큰을 사용하여 문장의 시작을 표시한다.
    while sent[-1] != "</s>":           # 매번 다음 단어를 예측해서 문장에 추가한다.
      prev = tuple(sent[-(self.n - 1):]) if self.n > 1 else() # 현재 시점에서 이전 (n - 1) 개의 단어를 뽑는다.
      candidates = [(ngram[-1], prob) for ngram, prob in self.model.items() if ngram[:-1] == prev] # 학습된 ngram 모델에서 ngram의 앞부분(n-1)이 prev와 같다면, 그 ngram의 마지막 단어를 candidates로 뽑는다.

      if not candidates:
        sent.append("</s>")
        break

      tokens, probs = zip(*candidates) # 대표로 뽑은 단어와 그에 해당하는 확률을 뽑는다.
      total = sum(probs)
      probs = [p / total for p in probs] # 확률을 전체에서 정규화한다.
      next_token = random.choices(tokens, weights = probs, k = 1)[0] # 확률에 따라서 뽑고 그것을 결과로 냅니다.

      if debug:
         print(f"{prev} -> {next_token}")

      sent.append(next_token)
      if len(sent) >= max_len:
        sent.append("</s>")

    sentence = ' '.join(sent)
    return sentence, self.perplexity_from_tokens(sent)

  def clean(self, raw_sentences):
    tokens = raw_sentences.split()
    cleaned = []

    for token in tokens:
      if token in ("<s>", "</s>"):
        continue
      if "/" in token:
        word = token.split("/")[0]
        cleaned.append(word)
      else:
        cleaned.append(token)

    return " ".join(cleaned)



In [54]:
### 4. 실행 ###
if __name__ == '__main__':
    from itertools import product

    # 하이퍼파라미터 설정
    n = 3
    laplace = 0.05
    num_sentences = 10

    # 데이터 불러오기
    print("데이터 로딩 중...")
    train_sentences = load_korean_data("ratings_train.txt")
    print(f"총 {len(train_sentences)}개의 문장 불러옴.")

    # 1만개만 선택
    random.shuffle(train_sentences)
    subset = train_sentences[:20000]

    # 모델 학습
    print(f"{n}-gram 모델 학습 중 (Laplace smoothing = {laplace})...")
    lm = LanguageModel(subset, n=n, laplace=laplace)

    # 문장 생성 및 평가
    print("\n생성된 문장들:")
    results = [lm.generate_sentence_from_seed(seed=i) for i in range(num_sentences)]
    for i, (s, ppl) in enumerate(results):
        cleaned = lm.clean(s)
        print(f"{i+1}. {cleaned} (Perplexity : {ppl : .3f})")
    # 가장 좋은 문장 선택
    best = min(results, key=lambda x: x[1])
    print("\n가장 자연스러운 문장:")
    print(f"{lm.clean(best[0])  } (Perplexity: {best[1]:.3f})")

데이터 로딩 중...
총 150000개의 문장 불러옴.
3-gram 모델 학습 중 (Laplace smoothing = 0.05)...

생성된 문장들:
1. 어메이징 스파이더맨 ! (Perplexity :  2730.786)
2. 휴잭맨 과 아이유 의 유쾌한 일본 유랑 기 혹은 망고 나무 아래 (Perplexity :  1157.907)
3. 봤더니 사랑 고 ㅏ 전쟁 ' 의 감독 이기도 하다니 .. (Perplexity :  1973.812)
4. 너무 귀여운 돼지 베 이브 ~!!! (Perplexity :  504.655)
5. 근거 없는 확신 이 지나고 나면 별일 아닌것 처럼 느껴지지만 그것 이 외면 할 수 없었음 !!! (Perplexity :  338.291)
6. 저 학교 1 층 에는 창문 도 없나 (Perplexity :  1477.127)
7. 20 살 까지의 인생 ' 그 이상 의 작품 ㅎㅎㅎ ^^ (Perplexity :  1809.119)
8. 참 .. 재미없다 .. (Perplexity :  153.756)
9. 평생토록 잊지못 할 .... (Perplexity :  901.703)
10. 매트릭스 의 꿈 의 영상 실록 (Perplexity :  280.990)

가장 자연스러운 문장:
참 .. 재미없다 .. (Perplexity: 153.756)


## **느낀 점**

- 이번 프로젝트를 통해 N-gram 모델을 구현해보며,최신의 대규모 언어모델보다는

  성능이 아쉽지만, 기초적인 언어 생성 모델의 작동 원리를 이해할 수 있었습니다.


- 특히 **Smoothing 기법**을 활용해 볼 수 있어 좋았습니다.

- 처음엔 한국어 문장분석이 어렵지 않을까 걱정했지만, 형태소 분석 라이브러리가 있어 수월하게 진행할 수 있었습니다.

  언제나 문장의 시작점과 끝점은 꼭 명시해주는 것이중요하다는 사실도 알았습니다.

- N-gram에서 이전 데이터들을 많이 포함할수록(n이 커질수록) 문장이 읽기가 좋아지는 편인 것 같습니다.

- 마지막으로, 단순히 코드를 복사/붙여넣기 하지 않고,
직접 주석을 달아보고, 코드를 이해하고, 오류를 개선하는 과정에서 많이 공부하였습니다.

- 생각보다 시간이 오래걸렸지만, 의미 있었던 토이프로젝트라고 생각합니다.